In [1]:
import numpy as np
import sympy as sp
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic
import math

In [2]:
def classRepresentatives(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if tr(Q)==np.round(trace)]

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def classCFReps(trace): # gives continued fractions of a given trace
    bound = math.floor(trace**2-4)
    cfList = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            cfList+=cfReps(D)
        elif D%4 == 1:
            cfList+=cfReps(D)
        i = i + 1
    return [cf for cf in cfList if cfTrace(cf)==np.round(trace)]

def cfReps(D):
    fracReps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                fracReps.append(frac)
            else:
                continue
    return fracReps

def classCFReps2(trace): # gives continued fractions of a given trace
    bound = math.floor(trace**2-4)
    cfList = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            cfList+=cfReps2(D)
        elif D%4 == 1:
            cfList+=cfReps2(D)
        i = i + 1
    return [cf for cf in cfList if cfTrace(cf)==np.round(trace)]

def cfReps2(D):
    fracReps = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            minRep = min(permutationsOf(frac))
            if minRep not in fracReps and gcd(A,B,C)==1 and min(permutationsOf(minRep[::-1])) != minRep:
                fracReps.append(minRep)
            else:
                continue
    return fracReps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

def cfTrace(cf):
    return np.trace(cfToMat(cf))

def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False

def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))

def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [3]:
def contFracToLR(contFrac):
    if len(contFrac)%2==1:
        contFrac = contFrac + contFrac
    output = []
    for i in range(len(contFrac)):
        if i%2==1: #if original cont'd frac is even length, making last one R/alternating is same as this (hopefully)
            for j in range(contFrac[i]):
                output.append(1)
        else:
            for j in range(contFrac[i]):
                output.append(0)
    return output

def recover2(ls): #sends a whole list to a cont'd fraction (not clear whether this is needed)
    output = []
    i=0
    while i<len(ls):
        counter=1
        while(i<len(ls)-1 and ls[i] == ls[i+1]):
            i+=1
            counter+=1
        i+=1
        output.append(counter)
    if (len(output)%2!=0):
        output[len(output)-1]+=output[0]
        output = output[1:]
    return output

def indices(lrSeq): #positions of R's (R=1)
    return [i for i in range(len(lrSeq)) if lrSeq[i]==1]

def pairs(lrSeq): #pairs of R's on which to split
    output = []
    for i in indices(lrSeq):
        for j in indices(lrSeq):
            if i==j:
                continue
            else:
                output.append((i,j))
    return output

def split(contFrac): #splits a cont'd fraction on every possible pair of R's into 2 groups of R's and L's (in 0's and 1's)
    output = []
    converted = contFracToLR(contFrac)
    n = len(converted)
    splitPoints = pairs(converted)
    for i,j in splitPoints:
        temp = converted[i:]+converted[:i]
        temp1 = temp[1:(j-i)%n]
        temp2 = inv(temp[(j-i)%n+1:]) #inv(temp[(j-i)%n+1:])
        output.append((tuple(temp1),tuple(temp2)))
    return output

def printCon(lrSeq): #R = 1, L = 0
    a = ''
    for i in lrSeq:
        if i==1:
            a+= 'R'
        else:
            a+= 'L'
    return a

def printConLs(ls):
    printList = []
    for L in ls:
        printList.append(printCon(L))
    return printList

#def getDict(contFrac):
#    output = []
#    fracs = []
#    for t in range(1+2*e):
#        tr = trace-e+t
#        fracs += [ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] for Q in classRepresentatives(tr)]
#    for f in fracs:
#        output += split(f)
#    return output

#def cfListToDict(cfList):
#    outputDict = {}
#    for cf in cfList:
#        for obj in split(cf):
#            if obj[0] in outputDict:
#                outputDict[obj[0]].add(obj[1])
#            else:
#                outputDict[obj[0]] = {obj[1]}
#    return outputDict

def cfListToDicts(cfList):
    forwardDict = {}
    invDict = {}
    for cf in cfList:
        for obj in split(cf):
            if obj[0] in forwardDict:
                forwardDict[obj[0]].add(obj[1])
            else:
                forwardDict[obj[0]] = {obj[1]}
            if obj[1] in invDict:
                invDict[obj[1]].add(obj[0])
            else:
                invDict[obj[1]] = {obj[0]}
    return forwardDict,invDict

def inv(lrSeq): #inverts a list of 0's and 1's (R's and L's)
    return tuple((np.array(lrSeq[::-1])-1)**2)

def pairToFrac(s1,s2):
    out = list(s1)+[1]+list(inv(s2))+[1]
    return tuple(min(permutationsOf(recover2(out))))

#def getPants(cfList): #this clearly doesn't work
#    allPairs = cfListToDict(cfList)
#    output,fracs = [],[]
#    for s0,s1 in allPairs:
#        temp1,temp2 = set([tuple(a[1]) for a in allPairs if a[0]==s0]), set([tuple(a[1]) for a in allPairs if a[0]==s1])
#        s  = temp1.intersection(temp2)
#        for s2 in s:
#            s2 = list(s2)
#            output.append([s0,s1,s2])
#            fracs = [pairToFrac(s0,inv(s1)),pairToFrac(s1,inv(s2)),pairToFrac(s2,inv(s0))]
#    return fracs

def getPants(cfList):
    dictPair = cfListToDicts(cfList)
    forwardDict = dictPair[0]
    invDict = dictPair[1]
    pants = set()
    for s0 in forwardDict:
        for s1 in forwardDict[s0]:
            if s1 in forwardDict and s0 in invDict:
                for s2 in forwardDict[s1].intersection(invDict[s0]):
                    y0,y1,y2 = pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)
                    if y0==y1 or y1==y2 or y0==y2:
                        continue
                    pants.add(frozenset((y0,y1,y2)))
    return pants

def pantsInTraceRange(traceMin,traceMax):
    cfList = []
    for trace in range(traceMin,traceMax+1):
        cfList += classCFReps(trace)
    return getPants(cfList)

def pantsInTraceRangeAndWithGivenCuff(traceMin,traceMax,cuttingSequence):
    allPants = pantsInTraceRange(traceMin,traceMax)
    pantsWithCuff = []
    for pants in allPants:
        if min(permutationsOf(cuttingSequence)) in pants:
            pantsWithCuff.append(pants)
    return pantsWithCuff

def boundaryCountForOrientedCuffs(traceMin,traceMax,cuttingSequencePositive): #i sort of screwed up orientation here
                                                                                #since surface is on the left and therefore
                                                                                #we prefer counterclockwise...
    cuttingSequenceNegative = cuttingSequencePositive[::-1]
    positiveOrientationPants = pantsInTraceRangeAndWithGivenCuff(traceMin,traceMax,cuttingSequencePositive)
    negativeOrientationPants = pantsInTraceRangeAndWithGivenCuff(traceMin,traceMax,cuttingSequenceNegative)
    return (len(positiveOrientationPants) - len(negativeOrientationPants))
            

def cfToMat(cf):
    mat = np.identity(2)
    length = 0
    for x in cf:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)


def pantsToGeodesics(pList):
    positives = list()
    for p in pList:
        for geo in p:
            minGeo = min(permutationsOf(geo))
            invGeo = min(permutationsOf(minGeo[::-1]))
            if minGeo in positives or invGeo in positives:
                continue
            else:
                positives.append(minGeo)
    return positives
                
def geosToEyes(geos):
    eyes = list()
    for geo in geos:
        for i in range(0,len(geo),2):
            a = list(geo[i:])+list(geo[:i])
            a = a[1:]
            a[0]-=1
            a[len(a)-1]-=1
            eyes.append((geo,tuple(a)))
    return eyes

def cuspInv(cusp):
    invCusp = list(cusp[::-1])
    if len(invCusp) == 1 and invCusp[0] <= 0:
        return tuple(invCusp)
    else:
        if invCusp[0] == 0:
            invCusp = invCusp[1:]
        else:
            invCusp = [0] + invCusp
        if invCusp[-1] == 0:
            invCusp = invCusp[:-1]
        else:
            invCusp = invCusp + [0]
        return tuple(invCusp)

def eyesToCusps(eList):
    output = list()
    for e in eList:
        eye = e[1]
        invEye = cuspInv(eye)
        if eye in output or invEye in output:
            continue
        else:
            output.append(eye)
    return output
                
def pantsAndEyesToMatrix(pantList, eyeList, posGeoList, posCuspList):
    
    geoIndexDict = {}
    cuspIndexDict = {}
    
    basisList = posGeoList + posCuspList
    
    for i in range(len(basisList)):
        if i < len(posGeoList):
            geoIndexDict[posGeoList[i]] = i 
        else:
            cuspIndexDict[posCuspList[i-len(posGeoList)]] = i
        
    colList = []
    for pants in pantList:
        col = np.zeros(len(basisList))
        for geo in pants:
            if geo in geoIndexDict:
                col[geoIndexDict[geo]] = 1
            else:
                col[geoIndexDict[min(permutationsOf(geo[::-1]))]] = -1
        colList.append(col)
    
    for eye in eyeList:
        col = np.zeros(len(basisList))
        
        if eye[0] in geoIndexDict:
            col[geoIndexDict[eye[0]]] = 1
        else:
            col[geoIndexDict[min(permutationsOf(eye[0][::-1]))]] = -1
        
        if eye[1] in cuspIndexDict:
            col[cuspIndexDict[eye[1]]] = 1
        else:
            col[cuspIndexDict[cuspInv(eye[1])]] = -1
        
        colList.append(col)
    
    return np.column_stack(colList)

In [4]:
pantSet = pantsInTraceRange(40,60)

In [5]:
posGeoList = pantsToGeodesics(pantSet)

In [6]:
eyeList = geosToEyes(posGeoList)

In [7]:
posCuspList = eyesToCusps(eyeList)

In [8]:
pantList = list(pantSet)

In [9]:
mat = pantsAndEyesToMatrix(pantList,eyeList,posGeoList,posCuspList)

In [10]:
from pulp import *
import numpy as np

EPS = 1e-3

""" Input """
A = mat
b = np.zeros(A.shape[0])

""" MIP """
print('hi1')
# Variables
x = np.empty(A.shape[1], dtype=object)
print(len(x))
for i in range(A.shape[1]):
    x[i] = LpVariable("x" + str(i), lowBound=0, upBound=None)
print('hi2')
# Problem
prob = LpProblem("prob", LpMinimize)
print('hi3')
print(np.sum(x))
# Objective
prob += np.sum(x)
print('hi4')
# Constraints
for row in range(A.shape[0]):
    prob += np.dot(A[row], x) == b[row]
print('hi5')
prob += np.sum(x) >= EPS  # forbid zero-vector

print(prob)

# Solve
status = prob.solve()
resultList = pantList+eyeList
resultStr = ""
solutions = sorted(zip(resultList, x), key=lambda x:value(x[1]), reverse=True)
for i in range(len(x)):
    if value(x[i]) != 0.0:
        resultStr += "+" + str(value(x[i])) + "*" + str(resultList[i])
print(resultStr)
l1min = value(np.sum(x))

hi1
1474
hi2
hi3
x0 + x1 + x10 + x100 + x1000 + x1001 + x1002 + x1003 + x1004 + x1005 + x1006 + x1007 + x1008 + x1009 + x101 + x1010 + x1011 + x1012 + x1013 + x1014 + x1015 + x1016 + x1017 + x1018 + x1019 + x102 + x1020 + x1021 + x1022 + x1023 + x1024 + x1025 + x1026 + x1027 + x1028 + x1029 + x103 + x1030 + x1031 + x1032 + x1033 + x1034 + x1035 + x1036 + x1037 + x1038 + x1039 + x104 + x1040 + x1041 + x1042 + x1043 + x1044 + x1045 + x1046 + x1047 + x1048 + x1049 + x105 + x1050 + x1051 + x1052 + x1053 + x1054 + x1055 + x1056 + x1057 + x1058 + x1059 + x106 + x1060 + x1061 + x1062 + x1063 + x1064 + x1065 + x1066 + x1067 + x1068 + x1069 + x107 + x1070 + x1071 + x1072 + x1073 + x1074 + x1075 + x1076 + x1077 + x1078 + x1079 + x108 + x1080 + x1081 + x1082 + x1083 + x1084 + x1085 + x1086 + x1087 + x1088 + x1089 + x109 + x1090 + x1091 + x1092 + x1093 + x1094 + x1095 + x1096 + x1097 + x1098 + x1099 + x11 + x110 + x1100 + x1101 + x1102 + x1103 + x1104 + x1105 + x1106 + x1107 + x1108 + x1109 + x111

hi4
hi5
prob:
MINIMIZE
1*x0 + 1*x1 + 1*x10 + 1*x100 + 1*x1000 + 1*x1001 + 1*x1002 + 1*x1003 + 1*x1004 + 1*x1005 + 1*x1006 + 1*x1007 + 1*x1008 + 1*x1009 + 1*x101 + 1*x1010 + 1*x1011 + 1*x1012 + 1*x1013 + 1*x1014 + 1*x1015 + 1*x1016 + 1*x1017 + 1*x1018 + 1*x1019 + 1*x102 + 1*x1020 + 1*x1021 + 1*x1022 + 1*x1023 + 1*x1024 + 1*x1025 + 1*x1026 + 1*x1027 + 1*x1028 + 1*x1029 + 1*x103 + 1*x1030 + 1*x1031 + 1*x1032 + 1*x1033 + 1*x1034 + 1*x1035 + 1*x1036 + 1*x1037 + 1*x1038 + 1*x1039 + 1*x104 + 1*x1040 + 1*x1041 + 1*x1042 + 1*x1043 + 1*x1044 + 1*x1045 + 1*x1046 + 1*x1047 + 1*x1048 + 1*x1049 + 1*x105 + 1*x1050 + 1*x1051 + 1*x1052 + 1*x1053 + 1*x1054 + 1*x1055 + 1*x1056 + 1*x1057 + 1*x1058 + 1*x1059 + 1*x106 + 1*x1060 + 1*x1061 + 1*x1062 + 1*x1063 + 1*x1064 + 1*x1065 + 1*x1066 + 1*x1067 + 1*x1068 + 1*x1069 + 1*x107 + 1*x1070 + 1*x1071 + 1*x1072 + 1*x1073 + 1*x1074 + 1*x1075 + 1*x1076 + 1*x1077 + 1*x1078 + 1*x1079 + 1*x108 + 1*x1080 + 1*x1081 + 1*x1082 + 1*x1083 + 1*x1084 + 1*x1085 + 1*x1086 + 1*x1

+2.0833333e-05*frozenset({(1, 2, 1, 11), (8, 6), (1, 1, 2, 1, 3, 1)})+2.0833333e-05*frozenset({(3, 16), (3, 1, 8, 1), (6, 8)})+4.1666667e-05*frozenset({(1, 2, 1, 11), (1, 1, 8, 2), (1, 1, 1, 1, 2, 3)})+6.25e-05*frozenset({(1, 2, 1, 10), (1, 2, 6, 2), (1, 1, 3, 1, 1, 2)})+2.0833333e-05*frozenset({(1, 2, 1, 11), (4, 1, 8, 1), (1, 7, 2, 1)})+2.0833333e-05*frozenset({(1, 1, 1, 2, 1, 4), (1, 1, 1, 1, 5, 1), (1, 1, 2, 8)})+6.25e-05*frozenset({(3, 16), (1, 1, 2, 1, 1, 3), (12, 4)})+2.0833333e-05*frozenset({(1, 1, 1, 1, 3, 2), (1, 2, 6, 2), (1, 4, 1, 8)})+8.3333333e-05*frozenset({(1, 5, 3, 1), (1, 1, 1, 16), (2, 1, 11, 1)})+2.0833333e-05*frozenset({(3, 2, 5, 1), (1, 3, 1, 8), (2, 1, 2, 5)})+8.3333333e-05*((16, 3), (1,))+2.0833333e-05*((1, 5, 2, 3), (2, 1, 4))+8.3333333e-05*((1, 1, 16, 1), (0, 1, 0))+2.0833333e-05*((1, 1, 2, 8), (7, 1, 0))+2.0833333e-05*((1, 1, 1, 3, 1, 2), (0, 1, 3, 1, 1))+8.3333333e-05*((1, 3, 5, 1), (2, 5, 0))+2.0833333e-05*((1, 2, 7, 1), (1, 7, 0))+2.0833333e-05*((1, 1, 4, 

In [108]:
print(l1min)

6.0


In [112]:
l2min = 0
for xi in x:
    l2min += value(xi)**2
l2min = np.sqrt(l2min)
print(l2min)

2.449489742783178


In [114]:
import cvxpy as cp
EPS = 0.5
# input
A = mat
b = np.zeros(A.shape[0])
# Construct a CVXPY problem
x = cp.Variable(A.shape[1], integer=True)
objective = cp.Minimize(cp.sum_squares(x))
constraints = []
for var in x:
    constraints.append(var >= 0)
constraints.append(A@x==b)
constraints.append(cp.sum(x) >= l1min)
constraints.append(cp.norm(x,2) <= l2min)

prob = cp.Problem(objective,constraints)
prob.solve()
resultList = pantList+eyeList
resultStr = ""
for i in range(len(x.value)):
    if round(x.value[i]) != 0.0:
        resultStr += "+" + str(round(x.value[i])) + "*" + str(resultList[i])
print(resultStr)

+1.0*frozenset({(1, 1, 1, 1, 5, 1), (1, 2, 1, 6), (4, 8)})+1.0*frozenset({(1, 2, 1, 10), (2, 1, 6, 1), (2, 1, 2, 5)})+1.0*((8, 4), (2,))+1.0*((1, 1, 1, 1, 1, 5), (0, 1, 5, 1, 0))+1.0*((2, 1, 10, 1), (0, 2, 0))+1.0*((1, 2, 5, 2), (1, 5, 1))


In [105]:
len(pantsInTraceRange(10,30))

1045

In [ ]:
#cvxpy with integer constraints

import cvxpy as cp
EPS = 0.5
# input
A = mat
b = np.zeros(A.shape[0])
# Construct a CVXPY problem
x = cp.Variable(A.shape[1], integer=True)
objective = cp.Minimize(cp.sum_squares(x))
constraints = []
for var in x:
    constraints.append(var >= 0)
constraints.append(A@x==b)
constraints.append(cp.sum(x) >= l1min)
constraints.append(cp.norm(x,2) <= l2min)

prob = cp.Problem(objective,constraints)
prob.solve()
resultList = pantList+eyeList
resultStr = ""
for i in range(len(x.value)):
    if round(x.value[i]) != 0.0:
        resultStr += "+" + str(round(x.value[i])) + "*" + str(resultList[i])
print(resultStr)

In [147]:
#cvxpy without integer constraints

import cvxpy as cp

# input
A = mat
b = np.zeros(A.shape[0])
# Construct a CVXPY problem
x = cp.Variable(A.shape[1],nonneg = True)
objective = cp.Minimize(cp.sum_squares(x))
constraints = []
constraints.append(A@x==b)
constraints.append(cp.sum(x) == 10**6) #scaled solutions

prob = cp.Problem(objective,constraints)
prob.solve()
resultList = pantList+eyeList
resultStr = ""
solList = list(zip(resultList,x.value))
solList.sort(key=lambda x:x[1], reverse=True)
for x in solList:
    resultStr += "+" + str(x[1]) + "*" + str(x[0]) + "\n"
print(resultStr)

+28619.20121249579*((1, 1, 15, 1), (0, 1, 0))
+25660.18769962574*((1, 1, 13, 1), (0, 1, 0))
+23894.765268796677*((1, 1, 11, 1), (0, 1, 0))
+20017.267678851505*((2, 1, 10, 1), (0, 2, 0))
+17510.99210177196*((2, 1, 9, 1), (0, 2, 0))
+15903.392920502298*((16, 3), (1,))
+15341.929216294142*((15, 3), (1,))
+14696.19120702554*((1, 1, 4, 3), (2, 1, 0))
+14371.31517795031*((12, 4), (2,))
+13861.000478740303*((14, 3), (1,))
+13183.274230244544*((10, 4), (2,))
+12664.21685873298*((2, 2, 4, 1), (0, 2, 1))
+12175.975634349581*((12, 3), (1,))
+11903.18939105167*((4, 1, 6, 1), (0, 4, 0))
+11889.22895808464*((8, 6), (4,))
+11770.144082637316*((1, 1, 1, 1, 3, 1), (0, 1, 1, 1, 0))
+11091.1574150822*((11, 3), (1,))
+9973.670378071458*((8, 4), (2,))
+9876.772173565936*((1, 1, 3, 5), (4, 1, 0))
+9800.69851622046*((10, 3), (1,))
+8898.18580055325*((4, 1, 6, 1), (0, 6, 0))
+8898.185778886764*((5, 8), (6,))
+8812.121959397322*((1, 6, 2, 1), (0, 1, 5))
+8607.488990715085*((1, 1, 2, 1, 1, 3), (0, 1, 3, 1, 0))


In [154]:
from pulp import *
import numpy as np

""" Input """
A = mat
b = np.zeros(A.shape[0])

""" MIP """
# Variables
x = np.empty(A.shape[1], dtype=object)
for i in range(A.shape[1]):
    x[i] = LpVariable("x" + str(i), lowBound=0, upBound=None)

# Problem
prob = LpProblem("prob", LpMinimize)

# Objective
prob += np.sum(x)

# Constraints
for row in range(A.shape[0]):
    prob += np.dot(A[row], x) == b[row]

prob += np.sum(x) == 10**6 # forbid zero-vector

# Solve
status = prob.solve()
resultList = pantList+eyeList
resultStr = ""
solList = list(zip(resultList,x))
solList.sort(key=lambda x:x[1], reverse=True)
print(resultStr)

TypeError: '<' not supported between instances of 'LpVariable' and 'LpVariable'

In [153]:
#cvxpy without integer constraints

import cvxpy as cp

# input
A = mat
b = np.zeros(A.shape[0])
# Construct a CVXPY problem
x = cp.Variable(A.shape[1],nonneg = True)
objective = cp.Minimize(cp.sum_squares(x))
constraints = []
constraints.append(A@x==b)
constraints.append(cp.sum(x) == 10**6) #scaled solutions

prob = cp.Problem(objective,constraints)
prob.solve()
resultList = pantList+eyeList
resultStr = ""
solList = list(zip(resultList,x.value))
solList.sort(key=lambda x:value(x[1]), reverse=True)
for x in solList:
    resultStr += "+" + str(x[1]) + "*" + str(x[0]) + "\n"
print(resultStr)

+28619.20121249579*((1, 1, 15, 1), (0, 1, 0))
+25660.18769962574*((1, 1, 13, 1), (0, 1, 0))
+23894.765268796677*((1, 1, 11, 1), (0, 1, 0))
+20017.267678851505*((2, 1, 10, 1), (0, 2, 0))
+17510.99210177196*((2, 1, 9, 1), (0, 2, 0))
+15903.392920502298*((16, 3), (1,))
+15341.929216294142*((15, 3), (1,))
+14696.19120702554*((1, 1, 4, 3), (2, 1, 0))
+14371.31517795031*((12, 4), (2,))
+13861.000478740303*((14, 3), (1,))
+13183.274230244544*((10, 4), (2,))
+12664.21685873298*((2, 2, 4, 1), (0, 2, 1))
+12175.975634349581*((12, 3), (1,))
+11903.18939105167*((4, 1, 6, 1), (0, 4, 0))
+11889.22895808464*((8, 6), (4,))
+11770.144082637316*((1, 1, 1, 1, 3, 1), (0, 1, 1, 1, 0))
+11091.1574150822*((11, 3), (1,))
+9973.670378071458*((8, 4), (2,))
+9876.772173565936*((1, 1, 3, 5), (4, 1, 0))
+9800.69851622046*((10, 3), (1,))
+8898.18580055325*((4, 1, 6, 1), (0, 6, 0))
+8898.185778886764*((5, 8), (6,))
+8812.121959397322*((1, 6, 2, 1), (0, 1, 5))
+8607.488990715085*((1, 1, 2, 1, 1, 3), (0, 1, 3, 1, 0))


[25.0,
 5.0,
 3.0,
 11.0,
 23.0,
 3.0,
 19.0,
 8.0,
 -8.0,
 12.0,
 23.0,
 -8.0,
 14.0,
 4.0,
 -3.0,
 6.0,
 10.0,
 9.0,
 3.0,
 5.0,
 22.0,
 -10.0,
 4.0,
 -3.0,
 -3.0,
 4.0,
 10.0,
 6.0,
 2.0,
 8.0,
 -17.0,
 -1.0,
 26.0,
 14.0,
 7.0,
 19.0,
 2.0,
 -7.0,
 -6.0,
 35.0,
 24.0,
 10.0,
 1.0,
 7.0,
 9.0,
 7.0,
 10.0,
 6.0,
 -3.0,
 8.0,
 -2.0,
 2.0,
 -2.0,
 7.0,
 3.0,
 12.0,
 3.0,
 25.0,
 -2.0,
 6.0,
 16.0,
 1.0,
 11.0,
 -1.0,
 22.0,
 4.0,
 7.0,
 6.0,
 -7.0,
 1.0,
 -14.0,
 10.0,
 11.0,
 16.0,
 13.0,
 -13.0,
 7.0,
 12.0,
 16.0,
 -1.0,
 -6.0,
 35.0,
 14.0,
 27.0,
 6.0,
 22.0,
 -2.0,
 4.0,
 4.0,
 10.0,
 0.0,
 5.0,
 -4.0,
 16.0,
 6.0,
 -15.0,
 -6.0,
 13.0,
 25.0,
 7.0,
 4.0,
 0.0,
 2.0,
 -10.0,
 5.0,
 -2.0,
 6.0,
 3.0,
 5.0,
 3.0,
 1.0,
 4.0,
 0.0,
 2.0,
 1.0,
 1.0,
 0.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 1.0,
 1.0,
 0.0,
 -1.0,
 0.0,
 2.0,
 2.0,
 0.0,
 2.0,
 1.0,
 2.0,
 -1.0,
 1.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 0.0,
 1.0,
 0.0,

In [184]:
curveList = posGeoList + posCuspList
closedLen = len(posGeoList)
mult = np.matmul(A,np.ones(A.shape[1]))
multCopy = np.matmul(np.abs(A),np.ones(A.shape[1]))
solList = list(zip(mult,multCopy,mult/multCopy,curveList))
for x in solList:
    print(x)

(-5.0, 27.0, -0.18518518518518517, (5, 26))
(14.0, 24.0, 0.5833333333333334, (1, 1, 4, 1, 1, 5))
(-2.0, 26.0, -0.07692307692307693, (22, 6))
(2.0, 22.0, 0.09090909090909091, (1, 2, 12, 3))
(6.0, 18.0, 0.3333333333333333, (1, 1, 2, 1, 1, 9))
(7.0, 23.0, 0.30434782608695654, (8, 15))
(3.0, 21.0, 0.14285714285714285, (21, 6))
(7.0, 21.0, 0.3333333333333333, (1, 1, 3, 2, 4, 1))
(2.0, 16.0, 0.125, (1, 3, 1, 24))
(6.0, 18.0, 0.3333333333333333, (3, 40))
(-5.0, 25.0, -0.2, (2, 1, 31, 1))
(5.0, 23.0, 0.21739130434782608, (1, 4, 6, 3))
(7.0, 17.0, 0.4117647058823529, (1, 8, 1, 12))
(6.0, 18.0, 0.3333333333333333, (1, 1, 10, 6))
(4.0, 18.0, 0.2222222222222222, (1, 1, 6, 1, 4, 1))
(1.0, 27.0, 0.037037037037037035, (1, 1, 6, 1, 2, 2))
(5.0, 11.0, 0.45454545454545453, (1, 1, 1, 41))
(-1.0, 21.0, -0.047619047619047616, (33, 4))
(11.0, 17.0, 0.6470588235294118, (1, 1, 1, 4, 5, 1))
(3.0, 21.0, 0.14285714285714285, (1, 7, 4, 3))
(2.0, 24.0, 0.08333333333333333, (1, 1, 3, 2, 1, 4))
(7.0, 17.0, 0.4117647

In [192]:
#cvxpy without integer constraints

import cvxpy as cp
resultList = pantList+eyeList
P = np.diag([1]*len(pantList) + [0]*len(eyeList))
# input
A = mat
b = np.zeros(A.shape[0])
# Construct a CVXPY problem
x = cp.Variable(A.shape[1],nonneg = True)
objective = cp.Minimize(cp.quad_form(x,P))
constraints = []
constraints.append(A@x==b)
constraints.append(cp.sum(x) == 10**6) #scaled solutions

prob = cp.Problem(objective,constraints)
prob.solve()
resultList = pantList+eyeList
resultStr = ""
solList = list(zip(resultList,x.value))
solList.sort(key=lambda x:value(x[1]), reverse=True)
for x in solList:
    resultStr += "+" + str(x[1]) + "*" + str(x[0]) + "\n"
print(resultStr)

+8304.121371187033*((1, 3, 1, 6, 2, 1), (2, 1, 6, 2, 0))
+8304.121268364375*((1, 3, 6, 1, 2, 1), (2, 6, 1, 2, 0))
+6737.1939975269*((3, 1, 18, 2), (1, 3, 0))
+6560.093649897388*((1, 3, 23, 1), (0, 1, 2))
+6490.90024599227*((1, 2, 4, 1, 5, 1), (0, 1, 2, 4, 0))
+6490.900211639038*((2, 2, 6, 5), (4, 2, 1))
+6479.523201356251*((3, 1, 33, 1), (0, 3, 0))
+6219.776404707343*((2, 1, 42, 1), (0, 2, 0))
+6219.776340342771*((40, 4), (2,))
+5915.4152278008105*((4, 2, 14, 1), (0, 4, 1))
+5915.415023802421*((1, 5, 14, 1), (0, 1, 4))
+5064.478243715061*((1, 2, 4, 1, 3, 2), (0, 3, 2, 1, 1))
+5064.477689006644*((1, 1, 2, 4, 4, 1), (0, 1, 1, 2, 3))
+4974.572082468461*((1, 2, 21, 2), (1, 1, 1))
+4974.572070783253*((1, 1, 1, 1, 21, 1), (0, 1, 1, 1, 0))
+4951.278721355884*((1, 1, 3, 2, 6, 1), (1, 6, 1, 1, 0))
+4951.278454461681*((1, 1, 3, 2, 1, 6), (1, 1, 6, 1, 0))
+4661.705555560247*((2, 2, 9, 3), (2, 2, 1))
+4661.705516613685*((1, 2, 2, 1, 9, 1), (0, 1, 2, 2, 0))
+4598.012306371461*((1, 1, 8, 2, 2, 1), (

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

[]